In [1]:
import pyodbc
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import acf, pacf, adfuller

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

import warnings
warnings.filterwarnings("ignore")

from pycaret.time_series import *
from datetime import datetime

## Funciones

#### Funcion para descargar los datos

In [2]:
def df_cluster(nits_clientes, fecha_final):
    # Conexion al dwh
    cnxn = pyodbc.connect(
        driver='{SQL Server}',
        server='192.168.100.58',
        uid='bilectura',
        pwd='D1sp@p3l3s')
    cursor = cnxn.cursor()

    df_SQL_nits = pd.DataFrame()

    for nit in nits_clientes:
        #Consulta SQL
        consulta_SQL =  f"SELECT DATEFROMPARTS(VTAANO, VTAMES, 1) AS 'Fecha', CONCAT(CONCAT(VTANIT, '-'), VTASUC) AS 'Nitcliente-sucursal',\
                        SUM(VTAVLRVTA) AS 'Ventas' FROM V_VTA_VTAHEC WHERE CONCAT(CONCAT(VTANIT, '-'), VTASUC) = '{nit}' AND VTAFCH < '{fecha_final}'\
                        GROUP BY DATEFROMPARTS(VTAANO, VTAMES, 1), CONCAT(CONCAT(VTANIT, '-'), VTASUC)"

        #Carga de la data desde el dwh de Dispapeles y se guarda en df
        cursor.execute(consulta_SQL)
        rows = cursor.fetchall()
        df_SQL_int = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])
        df_SQL_int["Ventas"] = df_SQL_int["Ventas"].astype(int)
        df_SQL_int["Fecha"] = pd.to_datetime(df_SQL_int["Fecha"])

        df_SQL_nits = pd.concat([df_SQL_nits, df_SQL_int], ignore_index= True)

    df_SQL = df_SQL_nits.groupby("Fecha").sum().reset_index()
    df_SQL_nits = df_SQL_nits.groupby("Nitcliente-sucursal").sum().reset_index()

    return df_SQL, df_SQL_nits

#### Funcion EDA

In [3]:
fig_kwargs = {
        "renderer": "png",
        "width": 1000,
        "height": 600,}

def lineplot(bd):
        x = bd["Fecha"]
        x_n = np.arange(0, len(bd))
        y = bd["Ventas"]   
        coeficientes = np.polyfit(x_n, y, 1)
        poli = np.poly1d(coeficientes)

        trace1 = go.Scatter(x=x, y=y, mode='lines+markers', name='Ventas')
        trace2 = go.Scatter(x=x, y=poli(x_n), mode='lines', name='Línea de Tendencia')

        layout = go.Layout(
                title='Ventas por mes',
                xaxis=dict(title='Fecha'),
                yaxis=dict(title='Ventas'),
                legend=dict(x=1, y=1)
        )

        fig = go.Figure()
        fig.add_trace(trace1)
        fig.add_trace(trace2)
        fig.update_layout(layout)
        fig.show()

def lineplot_diff(bd):
        ventas = bd["Ventas"]
        for i in range(1):
                ventas = np.diff(ventas)
        x = bd["Fecha"]
        y = ventas
        y_mean = np.repeat(np.mean(ventas), len(x))
        y_std_up = np.repeat(np.mean(ventas) + np.std(ventas), len(x))
        y_std_do = np.repeat(np.mean(ventas) - np.std(ventas), len(x))
        trace1 = go.Scatter(x=x, y=y, mode='lines+markers', name='Ventas')
        trace2 = go.Scatter(x=x, y=y_mean, mode='lines', name='Mean')
        trace3 = go.Scatter(x=x, y=y_std_up, mode='lines', name='StdUp')
        trace4 = go.Scatter(x=x, y=y_std_do, mode='lines', name='StdDown')
        layout = go.Layout(title='Diferencias',xaxis=dict(title='Fecha'),yaxis=dict(title='Diferencias'),legend=dict(x=1, y=1))

        fig = go.Figure()
        fig.add_trace(trace1)
        fig.add_trace(trace2)
        fig.add_trace(trace3)
        fig.add_trace(trace4)
        fig.update_layout(layout)
        fig.show()

def create_corr_plot(series, plot_pacf=False):
        corr_array = pacf(series.dropna(), alpha=0.05) if plot_pacf else acf(series.dropna(), alpha=0.05)
        lower_y = corr_array[1][:,0] - corr_array[0]
        upper_y = corr_array[1][:,1] - corr_array[0]

        # extraido de https://community.plotly.com/t/plot-pacf-plot-acf-autocorrelation-plot-and-lag-plot/24108/3
        fig = go.Figure()
        [fig.add_scatter(x=(x,x), y=(0,corr_array[0][x]), mode='lines',line_color='#3f3f3f') 
        for x in range(len(corr_array[0]))]
        fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4',
                        marker_size=12)
        fig.add_scatter(x=np.arange(len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
        fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines',fillcolor='rgba(32, 146, 230,0.3)',
                fill='tonexty', line_color='rgba(255,255,255,0)')
        fig.update_traces(showlegend=False)
        fig.update_xaxes(range=[-1,24])
        fig.update_yaxes(zerolinecolor='#000000')

        title='Partial Autocorrelation (PACF)' if plot_pacf else 'Autocorrelation (ACF)'
        fig.update_layout(title=title)
        fig.show()

def estadisticas_estacionalidad(bd):

        # Aplicar la prueba de Dickey-Fuller
        result = adfuller(bd["Ventas"])

        #Prueba Dickey-Fuller
        p_value = round(result[1], 4)

        print("Se realiza la prueba de Dickey-Fuller para analizar la estacionalidad de la serie de tiempo.")
        print("H0: la serie no es estacionaria")
        print("Con un alfa de 5% se determina:")
        #p_value
        if p_value < 0.05:
                print(f"Se rechaza la H0 ya que el p_valor de {p_value} es menor al alfa")
        else:
                print(f"No se tiene suficiente evidencia para rechazar la H0, un p_valor de {p_value} es mayor al alfa")
        print("Por otro lado, los valores criticos de la prueba son los siguientes:")
        print(f"El estadístico de prueba es: {round(result[0], 4)}")
        print(f"Y los valores críticos:")
        for key, value in result[4].items():
                print(f"\t{key}: {round(value, 4)}")

def EDA_cluster(bd, bd_nits):
        #Variables
        bd = bd.reset_index()
        bd_nits = bd_nits
        primer_fecha = datetime.utcfromtimestamp(bd.iloc[0, 1].timestamp())
        ultima_fecha = datetime.utcfromtimestamp(bd.iloc[-1, 1].timestamp())
        describe_bd = bd.describe().applymap("{:,.0f}".format)
        describe_bd_nits = bd_nits.describe().applymap("{:,.0f}".format)

        print(f"Esta base de datos tiene ventas de {len(bd)} meses,")
        print(f"empezando desde el {primer_fecha.strftime('%d-%m-%Y')}")
        print(f"y terminando el {ultima_fecha.strftime('%d-%m-%Y')}")
        print("La composicion estadistica de la base de datos es la siguiente:")
        print(describe_bd["Ventas"][1:])
        print(" ")

        print(f"Por otro lado, esta compuesto por ventas de {len(bd_nits)} clientes")
        print("Y asi se comporta estadisticamente asi:")
        print(describe_bd_nits["Ventas"][1:])
        print(" ")
        lineplot(bd)
        lineplot_diff(bd)
        create_corr_plot(bd["Ventas"], plot_pacf=False)
        create_corr_plot(bd["Ventas"], plot_pacf=True)
        estadisticas_estacionalidad(bd)

### Funcion prediccion ARIMA

In [17]:
def test_ARIMA(bd, order=(1, 1, 1), s_order=(2, 1, 1, 12), alpha=0.05, test_size=0.2):
    data = bd.set_index(keys="Fecha")
    time_series_data = data["Suavizado"]

    # Dividir los datos en conjuntos de entrenamiento y prueba
    train_size = int(len(time_series_data) * (1 - test_size))
    train_data = time_series_data[:train_size]
    test_data = time_series_data[train_size:]

    model = SARIMAX(train_data, order=order, seasonal_order=s_order)
    model_fit = model.fit()

    forecast_result = model_fit.get_forecast(steps=len(test_data), alpha=alpha)

    forecast = forecast_result.predicted_mean
    stderr = forecast_result.se_mean
    conf_int = np.round(forecast_result.conf_int(alpha=0.05))

    # Calcular métricas de evaluación
    mae = mean_absolute_error(test_data, forecast)
    mse = mean_squared_error(test_data, forecast)
    r2 = r2_score(test_data, forecast)
    mape = mean_absolute_percentage_error(test_data, forecast)

    date_index = pd.date_range(train_data.index[-1], periods=len(test_data) + 1, closed='right', freq="MS")
    forecast_df = pd.DataFrame({'Predicciones': forecast, 'Error estándar': stderr,
                                'Intervalo de confianza': conf_int.values.tolist()},
                                index=date_index)

    metrics = {'MSE': mse, 'MAPE': mape}

    return forecast_df, conf_int, order, s_order, metrics

def pred_ARIMA(bd, order= (1,1,1), s_order= (2,1,1,12), n_per= 12, alpha= 0.05):
    data = bd.set_index(keys= "Fecha")
    time_series_data = data["Suavizado"]
    model = SARIMAX(time_series_data, order= order, seasonal_order= s_order)
    model_fit = model.fit()

    forecast_result = model_fit.get_forecast(steps= n_per, alpha= alpha)

    forecast = forecast_result.predicted_mean
    stderr = forecast_result.se_mean
    conf_int = np.round(forecast_result.conf_int(alpha=0.05))

    date_index = pd.date_range(time_series_data.index[-1], periods= n_per+1, closed='right', freq= "MS")
    forecast_df = pd.DataFrame({'Predicciones': forecast, 'Error estándar': stderr, 
                                'Intervalo de confianza': conf_int.values.tolist()},
                                index=date_index)

    return forecast_df, conf_int, order, s_order

def plot_ARIMA(bd, bd_forecast, conf_int, order, s_order):
    bd = bd.set_index(keys= "Fecha")
    # Crear una figura con subplots
    fig = make_subplots(specs=[[{'secondary_y': True}]])

    # Linea valores originales
    fig.add_trace(go.Scatter(x=bd.index, y=bd["Suavizado"],
                            name='Serie original',
                            line=dict(color='blue')),
                            secondary_y=False)
    # Linea predicciones
    fig.add_trace(go.Scatter(x=bd_forecast.index, y=bd_forecast['Predicciones'],
                            name='Predicciones', 
                            line=dict(color='red')), 
                            secondary_y=False)
    # Linea limite inferior
    fig.add_trace(go.Scatter(x=bd_forecast.index, y=conf_int.iloc[:, 0], 
                            name='Límite inferior', 
                            line=dict(color='pink'), 
                            showlegend=False), 
                            secondary_y=False)
    # Linea limite superior
    fig.add_trace(go.Scatter(x=bd_forecast.index, y=conf_int.iloc[:, 1],
                            name='Límite superior',
                            line=dict(color='pink'), fill='tonexty',
                            fillcolor='rgba(255, 0, 0, 0.1)',
                            showlegend=False), secondary_y=False)

    fig.update_layout(title=f'Predicciones ARIMA{order}{s_order}', xaxis_title='Fecha', yaxis_title='Valor')

    fig.show()

### Funcion suavizacion

In [5]:
def plot_suavizacion(bd, alpha, tipo_suavizacion):
    # Visualizar la serie original y suavizada
    x = bd["Fecha"]
    y1 = bd["Ventas"]
    y2 = bd["Suavizado"]
    trace1 = go.Scatter(x= x, y= y1, mode= "lines+markers", name= "Ventas")
    trace2 = go.Scatter(x= x, y= y2, mode= "lines+markers", name= f"Suavizado alfa {alpha}")

    layout = go.Layout(
            title=f"Ventas por mes, suavizado con {tipo_suavizacion} y alpha {alpha}",
            xaxis=dict(title="Fecha"),
            yaxis=dict(title="Ventas"),
            legend=dict(x=1, y=1)
            )
    # fig = go.Figure()
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(trace1, secondary_y= False)
    fig.add_trace(trace2, secondary_y= True)
    fig.update_yaxes(title_text="Ventas", secondary_y=False)
    fig.update_yaxes(title_text="Suavizacion", secondary_y=True)
    fig.update_layout(layout)
    fig.show()

def suavizacion(bd, alpha= 0.2, tipo_suavizacion= "exponencial simple"):
    # Aplicar suavización exponencial simple
    if tipo_suavizacion == "exponencial simple":
        bd["Suavizado"] = bd["Ventas"].ewm(alpha=alpha).mean()
    elif tipo_suavizacion == None:
        bd["Suavizado"] = bd["Ventas"]
    elif tipo_suavizacion == "logaritmica":
        bd["Suavizado"] = np.log(bd["Ventas"]).ewm(alpha=alpha).mean()
    else:
        bd["Suavizado"] = np.sqrt(bd["Ventas"]).ewm(alpha=alpha).mean()
    
    if tipo_suavizacion == None:
        alpha = "NA"
    else:
        alpha = alpha

    plot_suavizacion(bd, alpha, tipo_suavizacion)
    
    return bd

def inversa_suavizacion(bd, tipo_suavizacion= "exponencial simple"):
    if tipo_suavizacion == "logaritmica":
        print(np.exp(bd))
    elif tipo_suavizacion == "raiz cuadrada":
        print(bd.apply(lambda x: x**2))
    else:
        print(bd)

## Extraccion, tranformacion y analisis

#### Carga de los clusters

In [6]:
df_clusters = pd.read_csv("C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Clustering 12-04-23.csv",
                            encoding= 'utf-8', decimal= ",", sep= ";")
col_eliminar = ["Escala R", "Escala M", "Escala F", "Distrito-Nombretipozona", "Cluster"]
df_clusters = df_clusters.drop(col_eliminar, axis= 1)

filtro_distrito = 10
filtro_tipozona = "Institucional"
filtro_cluster = "A"

df_clusters_f = df_clusters[
                            (df_clusters["Codigo distrito"] == filtro_distrito) &
                            (df_clusters["Nombre tipo zona"] == filtro_tipozona) &
                            (df_clusters["Letra cluster"] == filtro_cluster)
                            ]

#### EDA del cluster elegido

In [7]:
# Definicion del df a realizar EDA
df_clusters_EDA = df_clusters.groupby(["Codigo distrito", "Nombre tipo zona", "Letra cluster"]).agg({"Nit cliente-sucursal": np.size}).reset_index()

In [8]:
# Variables auxiliares
lista_nits = df_clusters_f["Nit cliente-sucursal"].tolist()
fecha_final = '2023-03-31'
fecha_final = datetime.strptime(fecha_final, '%Y-%m-%d').strftime('%Y-%m-%d')

In [9]:
#dfs para EDA y forecast
ventas_cluster, ventas_nits = df_cluster(nits_clientes= lista_nits, fecha_final= fecha_final)

In [10]:
EDA_cluster(ventas_cluster, ventas_nits)

Esta base de datos tiene ventas de 63 meses,
empezando desde el 01-01-2018
y terminando el 01-03-2023
La composicion estadistica de la base de datos es la siguiente:
mean    1,272,685,039
std       366,555,367
min       550,400,964
25%     1,022,225,995
50%     1,282,105,716
75%     1,508,243,564
max     2,081,385,853
Name: Ventas, dtype: object
 
Por otro lado, esta compuesto por ventas de 110 clientes
Y asi se comporta estadisticamente asi:
mean       728,901,431
std      2,595,353,536
min          2,642,600
25%         22,790,312
50%         79,452,530
75%        301,514,128
max     24,380,149,839
Name: Ventas, dtype: object
 


Se realiza la prueba de Dickey-Fuller para analizar la estacionalidad de la serie de tiempo.
H0: la serie no es estacionaria
Con un alfa de 5% se determina:
No se tiene suficiente evidencia para rechazar la H0, un p_valor de 0.6734 es mayor al alfa
Por otro lado, los valores criticos de la prueba son los siguientes:
El estadístico de prueba es: -1.2004
Y los valores críticos:
	1%: -3.5485
	5%: -2.9128
	10%: -2.5941


##### Suavización

In [16]:
# El tipo de suavizacion puede ser:
    #Exponencial simple
    #Logaritmica
    #Raiz cuadrada
    #None
ventas_cluster = suavizacion(ventas_cluster, alpha= 0.2, tipo_suavizacion= None)

### Prediction

#### Fase de test de los paramestros del ARIMA

In [18]:
forecast_df, int_conf, order, s_order, metricas = test_ARIMA(bd= ventas_cluster, order= (6,1,6), s_order= (2,1,1,12), alpha= (0.05), test_size= 0.2)
print(metricas)
plot_ARIMA(bd= ventas_cluster, bd_forecast= forecast_df, conf_int= int_conf, order= order, s_order= s_order)

{'MSE': 4.367119036158158e+16, 'MAPE': 0.10203684212057909}


#### Proyeccion y graficas finales con los paramestros ajustados

In [13]:
forecast_df, int_conf, order, s_order = pred_ARIMA(bd= ventas_cluster, order= (6,1,6), s_order= (2,1,1,12), n_per= 12, alpha= (0.05))
plot_ARIMA(bd= ventas_cluster, bd_forecast= forecast_df, conf_int= int_conf, order= order, s_order= s_order)

In [14]:
pd.options.display.float_format = '{:,.0f}'.format
forecast_df

,Predicciones,Error estándar,Intervalo de confianza
2023-04-01,"1,829,369,501","143,187,334","[1548727482.0, 2110011519.0]"
2023-05-01,"1,809,896,863","165,584,188","[1485357818.0, 2134435908.0]"
2023-06-01,"1,892,397,620","167,200,473","[1564690714.0, 2220104526.0]"
2023-07-01,"1,978,651,431","203,818,295","[1579174914.0, 2378127949.0]"
2023-08-01,"1,991,838,727","213,234,489","[1573906807.0, 2409770646.0]"
2023-09-01,"1,954,070,240","232,575,544","[1498230550.0, 2409909929.0]"
2023-10-01,"1,959,556,201","246,266,550","[1476882633.0, 2442229770.0]"
2023-11-01,"2,098,837,701","251,060,908","[1606767364.0, 2590908038.0]"
2023-12-01,"2,254,081,629","259,539,873","[1745392825.0, 2762770434.0]"
2024-01-01,"1,845,337,540","265,919,475","[1324144945.0, 2366530134.0]"
